In [3]:
%pip install spacy==3.6.1
%pip install ml-datasets
!python -m spacy download en_core_web_md

     ---------------------------------------- 12.0/12.0 MB 6.5 MB/s eta 0:00:00
  Using cached preshed-3.0.9-cp310-cp310-win_amd64.whl (122 kB)
  Using cached pathy-0.10.2-py3-none-any.whl (48 kB)
  Using cached murmurhash-1.0.10-cp310-cp310-win_amd64.whl (25 kB)
  Using cached cymem-2.0.8-cp310-cp310-win_amd64.whl (39 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 9.4 MB/s eta 0:00:00
  Using cached blis-0.7.11-cp310-cp310-win_amd64.whl (6.6 MB)
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 42.8/42.8 MB 5.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [84]:
import spacy
import numpy as np
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

In [64]:
nlp = spacy.load("en_core_web_md") # может занять 5 минут

In [92]:
df=pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')
df=df[df['sentiment']!='neautral']
#train_data, valid_data = train_test_split(df,test_size=0.3, random_state=42)
train, val, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [56]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [122]:
nlp.pipe(train, as_tuples=True)

<generator object Language.pipe at 0x0000015BA6F9D230>

In [134]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        # One Hot Encodding
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [142]:
train=train.to_numpy()
val = val.to_numpy()
test = test.to_numpy()

In [139]:
# это для примера, можно взять больше количество текстов
train_docs = make_docs(train)

  0%|          | 0/36000 [00:00<?, ?it/s]

In [143]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(val)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/12000 [00:00<?, ?it/s]

In [144]:
# что здесь происходит: мы заполнили base_config, а эта команда на его основе прописывает config.cfg
! python -m spacy init fill-config base_config.cfg config.cfg
# после запуска этой команды нужно уменьшить max_steps и прописать пути train и dev

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [145]:
# тренируем модель
! python -m spacy train config.cfg --output ./output
# данных мало, сильно на метрики внимания не обращаем

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       36.08    0.36
  0     200         36.90       85.97    0.86
  0     400         25.67       87.64    0.88
  0     600         22.00       88.64    0.89
  0     800         18.11       90.02    0.90
  0    1000         17.37       90.93    0.91
  0    1200         17.41       91.71    0.92
  0    1400         13.56       91.69    0.92
  0    1600         14.10       91.89    0.92
  0    1800         12.98       92.02    0.92
  0    2000         13.98       92.26    0.92
  0    2200         12.13       92.88    0.93
  0    2400         12.40  

In [146]:
# загружаем лучшую модель
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while True:
    text = input("Please enter example input: ") # не обязательно в начальной форме
    if text == "quit":
        break
    print(text)
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")
        
# значения в пределах 0.5, модель не сильно уверена, нужно больше данных

type : ‘quit’ to exit
Please enter example input: грязная
грязная
{'positive': 0.21902033686637878, 'negative': 0.7809796929359436}
the sentiment is negative
Please enter example input: приятно сидит
приятно сидит
{'positive': 0.6716068983078003, 'negative': 0.3283931016921997}
the sentiment is positive
Please enter example input: норм 
норм 
{'positive': 0.7977607846260071, 'negative': 0.2022392451763153}
the sentiment is positive
Please enter example input: не очень
не очень
{'positive': 0.24931465089321136, 'negative': 0.7506853938102722}
the sentiment is negative
Please enter example input: quit
